In [1]:
import os
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import ElementNotInteractableException
from bs4 import BeautifulSoup
options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('lang=ko_KR')
chromedriver_path = "chromedriver"
driver = webdriver.Chrome(os.path.join(os.getcwd(), chromedriver_path), options=options)  # chromedriver 열기
def main():
    global driver, menu_wb
    driver.implicitly_wait(4)  # 렌더링 될때까지 기다린다 4초
    driver.get('https://map.kakao.com/')  # 주소 가져오기
    search("스타벅스 더종로R점")
    driver.quit()
    print("finish")
def search(place):
    global driver
    search_area = driver.find_element_by_xpath('//*[@id="search.keyword.query"]')  # 검색 창
    search_area.send_keys(place)  # 검색어 입력
    driver.find_element_by_xpath('//*[@id="search.keyword.submit"]').send_keys(Keys.ENTER)  # Enter로 검색
    sleep(1)
    # 검색된 정보가 있는 경우에만 탐색
    # 1번 페이지 place list 읽기
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    place_lists = soup.select('.placelist > .PlaceItem') # 검색된 장소 목록
    # 검색된 첫 페이지 장소 목록 크롤링하기
    crawling(place_lists)
    search_area.clear()
    # 우선 더보기 클릭해서 2페이지
    try:
        driver.find_element_by_xpath('//*[@id="info.search.place.more"]').send_keys(Keys.ENTER)
        sleep(1)
        # 2~ 5페이지 읽기
        for i in range(2, 6):
            # 페이지 넘기기
            xPath = '//*[@id="info.search.page.no' + str(i) + '"]'
            driver.find_element_by_xpath(xPath).send_keys(Keys.ENTER)
            sleep(1)
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            place_lists = soup.select('.placelist > .PlaceItem') # 장소 목록 list
            crawling(place_lists)
    except ElementNotInteractableException:
        print('not found')
    finally:
        search_area.clear()
def crawling(placeLists):
    for i, place in enumerate(placeLists):
        menuInfos = getMenuInfo(i, driver)
        print(menuInfos)
def getMenuInfo(i, driver):
    # 상세페이지로 가서 메뉴찾기
    detail_page_xpath = '//*[@id="info.search.place.list"]/li[' + str(i + 1) + ']/div[5]/div[4]/a[1]'
    driver.find_element_by_xpath(detail_page_xpath).send_keys(Keys.ENTER)
    driver.switch_to.window(driver.window_handles[-1])  # 상세정보 탭으로 변환
    sleep(1)
    menuInfos = []
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    # 메뉴의 3가지 타입
    menuonlyType = soup.select('.cont_menu > .list_menu > .menuonly_type')
    nophotoType = soup.select('.cont_menu > .list_menu > .nophoto_type')
    photoType = soup.select('.cont_menu > .list_menu > .photo_type')
    if len(menuonlyType) != 0:
        for menu in menuonlyType:
            menuInfos.append(_getMenuInfo(menu))
    elif len(nophotoType) != 0:
        for menu in nophotoType:
            menuInfos.append(_getMenuInfo(menu))
    else:
        for menu in photoType:
            menuInfos.append(_getMenuInfo(menu))
    driver.close()
    driver.switch_to.window(driver.window_handles[0])  # 검색 탭으로 전환
    return menuInfos
def _getMenuInfo(menu):
    menuName = menu.select('.info_menu > .loss_word')[0].text + '(Tall, Grande, Venti)'
    menuPrices = menu.select('.info_menu > .price_menu')
    menuPrice = ''
    menuDetail = menu.select('.info_menu > .txt_menu')[0].text
    if len(menuPrices) != 0:
        menuPrice =  menuPrices[0].text.split(' ')[1]
    return [menuName, menuPrice.replace("," ,""), int(menuPrice.replace("," ,""))+500, int(menuPrice.replace("," ,"")) +1000, menuDetail.replace("\n" ,"")]
if __name__ == "__main__":
    main()

[['나이트로 바닐라 크림(Tall, Grande, Venti)', '5900', 6400, 6900, '부드러운 목넘김의 나이트로 커피와 바닐라 크림의 매력을 한번에 느껴보세요!'], ['제주 샤이닝 바나나 라떼(Tall, Grande, Venti)', '7200', 7700, 8200, '[제주지역 한정음료] 반짝반짝 빛나는 제주를 상징하는 음료로 달콤한 바나나 소스를 활용한 제주 한정 Coffee 음료를 제주에서 즐겨보세요.'], ['제주 핑크 뮬리 밀크 티(Tall, Grande, Venti)', '6800', 7300, 7800, '[제주지역 한정음료] 로맨틱한 핑크빛과 보라빛의 핑크뮬리를형상화한 음료로 국내산 고구마와 블랙티의 조화.부드럽고 가볍게 마실 수 있는 고구마 밀크 티 음료를제주에서 경험해보세요!'], ['아이스 제주 핑크 뮬리 밀크 티(Tall, Grande, Venti)', '6800', 7300, 7800, '[제주지역 한정음료] 로맨틱한 핑크빛과 보라빛의 핑크뮬리를형상화한 음료로 국내산 고구마와 블랙티의 조화.부드럽고 가볍게 마실 수 있는 고구마 밀크 티 음료를제주에서 경험해보세요!'], ['아이스 제주 샤이닝 바나나 라떼(Tall, Grande, Venti)', '7200', 7700, 8200, '[제주지역 한정음료] 반짝반짝 빛나는 제주를 상징하는 음료로 달콤한 바나나 소스를 활용한 제주 한정 Coffee 음료를 제주에서 즐겨보세요.'], ['제주 샤이닝 바나나 크림 프라푸치노(Tall, Grande, Venti)', '7500', 8000, 8500, '[제주지역 한정음료] 반짝반짝 빛나는 제주를 상징하는 음료로 달콤한 바나나 소스를 활용한 제주 한정 음료를 더욱 시원하게 즐겨보세요.'], ['제주 별빛 애플 주스(Tall, Grande, Venti)', '6800', 7300, 7800, '[제주지역 한정음료] 제주의 별똥별과 밤하늘의 느낌을 담은 음료로 상큼한 사과착즙주스와 은은한 진저향을 느낄 수 있는 따뜻한 과일 주스.